**Merging transactions**

In [1]:
import pandas as pd
import numpy as np

'''Only to be used if "all_tranformations.csv" does not exist.'''
# df = pd.read_csv("data/transactions_1.csv")
# df2 = pd.read_csv("data/transactions_2.csv")
# df3 = pd.read_csv("data/transactions_3.csv")

# #combining datasets in one CSV
# transactions=pd.concat([df,df2,df3], ignore_index = True)

# #saving to directory
# transactions.to_csv(r'data/all_transactions.csv', index = False)

'Only to be used if "all_tranformations.csv" does not exist.'

**Merging datagrames**

In [2]:
df = pd.read_csv("data/all_transactions.csv")
df2 = pd.read_csv("data/users.csv")
df3 = pd.read_csv("data/devices.csv")
df4 = pd.merge(df2, df3[df3.brand != "Unknown"], on='user_id')
data = pd.merge(df4, df, on='user_id')
# df = pd.merge(df, df3, on='user_id')
# data = pd.merge(df, df2, on='user_id')

**Renaming columns**

In [3]:
data = data.rename(columns={"created_date_x": "user_created_date", "created_date_y": "transaction_created_date", "country": "user_country"})

**Removing unnecessary columns**

In [4]:
data = data.drop(['city', 'attributes_notifications_marketing_push', 'attributes_notifications_marketing_email', 'num_successful_referrals', 'ea_merchant_mcc', 'ea_merchant_city', 'ea_merchant_country', 'num_referrals'], axis=1)

**Column Tranformations**

In [5]:
plan_ = {"STANDARD": 0,"SILVER": 1,"GOLD": 1}
data['plan'] = data['plan'].map(plan_)

In [6]:
currency_ = {'AED' : 0, 'SEK' : 0, 'AUD' : 0, 'GBP' : 0, 'ETH' : 1, 'RUB' : 0, 'CHF' : 0, 'HRK' : 0, 'LTC' : 1, 'MAD' : 0, 'BTC' : 1, 'NZD' : 0, 'JPY' : 0, 'ILS' : 0, 'QAR' : 0, 'MXN' : 0, 'DKK' : 0, 'SGD' : 0, 'ZAR' : 0, 'BGN' : 0, 'USD' : 0, 'INR' : 0, 'THB' : 0, 'RON' : 0, 'HUF' : 0, 'TRY' : 0, 'XRP' : 1, 'PLN' : 0, 'EUR' : 0, 'BCH' : 1, 'CZK' : 0, 'CAD' : 0, 'NOK' : 0, 'HKD' : 0, 'SAR' : 0}
data['transactions_currency'] = data['transactions_currency'].map(currency_)

In [7]:
currency_ = {'FAILED': 0, 'DECLINED': 0, 'COMPLETED': 1, 'REVERTED': 2, 'PENDING': 0, 'CANCELLED': 0}
data['transactions_state'] = data['transactions_state'].map(currency_)

In [8]:
brands = {"Android": 0,"Apple": 1}
data['brand'] = data['brand'].map(brands)

In [9]:
continents_ = {'ES':'SWEU', 'LT':'EEU', 'IE':'WEU', 'GB':'WEU', 'MT':'SWEU', 'FR':'SWEU', 'RO':'SWEU', 'PL':'EEU', 'AT':'CEU', 'IT':'SWEU', 'SI':'CEU', 'CZ':'CEU', 'BE':'WEU', 'BG':'SWEU', 'GI':'SWEU', 'CH':'AS', 'GG':'SWEU', 'CY':'SWEU', 'DE':'CEU', 'SK':'CEU', 'GR':'SEU', 'DK':'NEU', 'PT':'SWEU', 'LU':'NEU', 'HR':'SWEU', 'NL':'NEU', 'SE':'NEU', 'LV':'EEU', 'IM':'NEU', 'FI':'NEU', 'EE':'EEU', 'JE':'SWEU', 'HU':'CEU', 'NO':'NEU', 'RE':'AF', 'IS':'NEU', 'GP':'NA', 'AU':'OC', 'LI':'EEU', 'MQ':'NA'}
data['user_country'] = data['user_country'].map(continents_)
data = data.rename(columns={"user_country": "continent"})
new_continents_ = {'OC':'rest_of_the_world','NA':'rest_of_the_world','AF':'rest_of_the_world','SEU':'rest_of_the_world','SWEU':'Central_Europe','CEU':'Central_Europe','NEU':'Central_Europe', 'WEU':'Western_Europe'}
data['continent'] = data['continent'].map(new_continents_)

In [10]:
data = data[(data.transactions_state == 1) | (data.transactions_state == 2)]

**Plots**

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

**Aggregating**

In [12]:
def age_bands(year):
    year = 2020 - year
    if year < 25:
        return 0
    elif year < 35:
        return 1
    elif year < 45:
        return 2
    elif year < 55:
        return 3
    elif year < 65:
        return 4
    return 5

In [13]:
# Demographics
users_agg_df = data.groupby(['user_id']).agg({'birth_year': 'first', 'continent': 'first', 'user_created_date': 'first', 'user_settings_crypto_unlocked': 'first', 'plan': 'first', 'num_contacts': 'first', 'brand': 'first'}).reset_index()
users_agg_df["birth_year"] = users_agg_df["birth_year"].apply(lambda x: age_bands(x))
users_agg_df

,user_id,birth_year,continent,user_created_date,user_settings_crypto_unlocked,plan,num_contacts,brand
0,user_0,1,Central_Europe,2018-09-10 18:46:42.843855,0,0,3,1
1,user_1,2,NaN,2018-01-11 18:53:10.531146,1,0,75,1
2,user_10,0,NaN,2018-01-26 18:40:43.166413,1,0,19,1
3,user_100,1,Central_Europe,2018-12-02 14:18:09.034151,0,0,0,0
4,user_1000,1,Central_Europe,2018-03-22 22:08:56.267644,0,1,13,1
...,...,...,...,...,...,...,...,...
14902,user_9995,1,NaN,2018-06-05 06:16:51.861843,0,0,0,1
14903,user_9996,4,Western_Europe,2018-10-26 03:59:55.030756,0,0,0,1
14904,user_9997,1,Western_Europe,2018-08-18 04:07:31.402857,0,0,6,0
14905,user_9998,1,NaN,2019-01-02 02:17:00.962571,0,0,21,0


In [14]:
# First and last transaction of each user
first_transactions_date_agg = data.groupby(["user_id"]).agg({'transaction_created_date': 'min'}).reset_index().rename(columns={"transaction_created_date": "first_transaction_date"})
last_transactions_date_agg = data.groupby(["user_id"]).agg({'transaction_created_date': 'max'}).reset_index().rename(columns={"transaction_created_date": "last_transaction_date"})
transactions_date_agg = pd.merge(first_transactions_date_agg, last_transactions_date_agg, on="user_id")
transactions_date_agg

,user_id,first_transaction_date,last_transaction_date
0,user_0,2018-09-10 18:51:20.218855,2019-05-10 05:09:13.868855
1,user_1,2018-02-04 08:27:53.463146,2019-05-04 02:30:11.492146
2,user_10,2018-06-29 15:37:38.050413,2019-05-11 13:07:42.624413
3,user_100,2018-12-02 14:22:58.423151,2019-02-24 13:36:50.054151
4,user_1000,2019-03-27 11:03:23.105644,2019-05-05 19:08:42.183644
...,...,...,...
14902,user_9995,2018-06-16 19:51:44.248843,2019-05-01 12:21:34.372843
14903,user_9996,2018-10-26 04:02:43.183756,2019-01-08 05:59:32.103756
14904,user_9997,2018-08-18 04:15:20.234857,2019-04-27 00:32:53.447857
14905,user_9998,2019-01-02 02:23:38.707571,2019-03-15 09:27:13.108571


In [15]:
# Total transactions
user_transactions = data.groupby(["user_id"]).agg({'transaction_created_date': 'count'}).reset_index().rename(columns={"transaction_created_date": "n_transactions"})
user_transactions

,user_id,n_transactions
0,user_0,528
1,user_1,139
2,user_10,126
3,user_100,58
4,user_1000,43
...,...,...
14902,user_9995,6
14903,user_9996,32
14904,user_9997,136
14905,user_9998,31


In [16]:
# Total amounts of each user
transactions_total_agg = data[data.transactions_state == 1].groupby(["user_id"]).agg({'amount_usd': 'sum'}).reset_index()
transactions_total_agg = transactions_total_agg.rename(columns={"amount_usd": "amount_total"})
transactions_total_rev = data[data.transactions_state == 2].groupby(["user_id"]).agg({'amount_usd': 'sum'}).reset_index().rename(columns={'amount_usd': 'amount_reverted'})
transactions_total_agg = pd.merge(transactions_total_agg, transactions_total_rev, on="user_id", how="left").fillna(0.0)
transactions_total_agg["total_amount"] = transactions_total_agg["amount_total"] - transactions_total_agg["amount_reverted"]
transactions_total_agg = transactions_total_agg[["user_id", "total_amount"]]
transactions_total_agg

,user_id,total_amount
0,user_0,6892.45
1,user_1,12953.66
2,user_10,5381.07
3,user_100,3716.07
4,user_1000,843.12
...,...,...
14784,user_9995,239.65
14785,user_9996,1680.76
14786,user_9997,4046.05
14787,user_9998,520.52


In [22]:
# Final df for classification
classification_df = pd.merge(users_agg_df, transactions_date_agg, on='user_id')
classification_df = pd.merge(classification_df, transactions_total_agg, on='user_id')
classification_df = pd.merge(classification_df, user_transactions, on='user_id')
dum = pd.get_dummies(classification_df.continent)
classification_df = pd.concat([classification_df, dum], axis=1)
classification_df['label'] = classification_df['plan']
classification_df = classification_df.drop(columns=['continent', 'plan',])
classification_df

,user_id,birth_year,user_created_date,user_settings_crypto_unlocked,num_contacts,brand,first_transaction_date,last_transaction_date,total_amount,n_transactions,Central_Europe,Western_Europe,rest_of_the_world,label
0,user_0,1,2018-09-10 18:46:42.843855,0,3,1,2018-09-10 18:51:20.218855,2019-05-10 05:09:13.868855,6892.45,528,1,0,0,0
1,user_1,2,2018-01-11 18:53:10.531146,1,75,1,2018-02-04 08:27:53.463146,2019-05-04 02:30:11.492146,12953.66,139,0,0,0,0
2,user_10,0,2018-01-26 18:40:43.166413,1,19,1,2018-06-29 15:37:38.050413,2019-05-11 13:07:42.624413,5381.07,126,0,0,0,0
3,user_100,1,2018-12-02 14:18:09.034151,0,0,0,2018-12-02 14:22:58.423151,2019-02-24 13:36:50.054151,3716.07,58,1,0,0,0
4,user_1000,1,2018-03-22 22:08:56.267644,0,13,1,2019-03-27 11:03:23.105644,2019-05-05 19:08:42.183644,843.12,43,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14784,user_9995,1,2018-06-05 06:16:51.861843,0,0,1,2018-06-16 19:51:44.248843,2019-05-01 12:21:34.372843,239.65,6,0,0,0,0
14785,user_9996,4,2018-10-26 03:59:55.030756,0,0,1,2018-10-26 04:02:43.183756,2019-01-08 05:59:32.103756,1680.76,32,0,1,0,0
14786,user_9997,1,2018-08-18 04:07:31.402857,0,6,0,2018-08-18 04:15:20.234857,2019-04-27 00:32:53.447857,4046.05,136,0,1,0,0
14787,user_9998,1,2019-01-02 02:17:00.962571,0,21,0,2019-01-02 02:23:38.707571,2019-03-15 09:27:13.108571,520.52,31,0,0,0,0


**Tests and raw code**

In [12]:
len(data[(data.transactions_state == 2) & (data.direction=="INBOUND")])

34575

In [38]:
pd.crosstab(index=data['transactions_type'],columns=data['transactions_state'])

transactions_state,0,1,2
transactions_type,,,
ATM,11781,60979,769
CARD_PAYMENT,121161,992555,51690
CARD_REFUND,1,9095,0
CASHBACK,4752,59964,0
EXCHANGE,0,124164,0
FEE,0,18671,0
REFUND,0,1175,0
TAX,14,2242,11
TOPUP,32145,241183,34575


In [37]:
pd.crosstab(index=data['transactions_type'],columns=data['direction'])

direction,INBOUND,OUTBOUND
transactions_type,,
ATM,0,73529
CARD_PAYMENT,0,1165406
CARD_REFUND,9096,0
CASHBACK,64716,0
EXCHANGE,0,124164
FEE,0,18671
REFUND,1175,0
TAX,0,2267
TOPUP,307903,0


In [39]:
pd.crosstab(index=data['direction'],columns=data['transactions_state'])

transactions_state,0,1,2
direction,,,
INBOUND,37471,347451,34575
OUTBOUND,136429,1555976,54194


In [33]:
data[(data.user_id == "user_7") & (data.transactions_state == 2) & (data.amount_usd == 8.51) & (data.direction == "INBOUND")]

,user_id,birth_year,continent,user_created_date,user_settings_crypto_unlocked,plan,num_contacts,brand,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,direction,transaction_created_date
1427,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_449647,TOPUP,0,8.51,2,NaN,INBOUND,2018-05-06 03:09:15.266388
1431,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_450581,TOPUP,0,8.51,2,NaN,INBOUND,2018-06-04 00:29:53.279388
1434,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_452103,TOPUP,0,8.51,2,NaN,INBOUND,2018-04-30 04:34:36.303388
1438,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_454996,TOPUP,0,8.51,2,NaN,INBOUND,2018-05-17 18:55:28.818388
1443,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_455627,TOPUP,0,8.51,2,NaN,INBOUND,2018-06-04 00:34:14.155388
1518,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_630524,TOPUP,0,8.51,2,NaN,INBOUND,2018-11-23 18:25:51.921388
1524,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_642824,TOPUP,0,8.51,2,NaN,INBOUND,2018-12-16 23:34:02.979388
1579,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_701891,TOPUP,0,8.51,2,NaN,INBOUND,2019-02-05 19:47:07.123388
1580,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_701895,TOPUP,0,8.51,2,NaN,INBOUND,2019-02-05 19:46:12.704388
1626,user_7,1992,SWEU,2018-03-08 06:18:43.016388,0,0,10,0,transaction_763158,TOPUP,0,8.51,2,NaN,INBOUND,2019-03-28 23:16:47.185388


In [50]:
data[data["user_id"] == "user_12038"].sort_values(by='transaction_created_date', ascending=True)

,user_id,birth_year,continent,user_created_date,user_settings_crypto_unlocked,plan,num_contacts,brand,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,direction,transaction_created_date
1694794,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1531201,TOPUP,0,0.18,2,NaN,INBOUND,2018-10-18 02:47:18.908703
1694796,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1554970,TOPUP,0,8.95,1,NaN,INBOUND,2018-10-18 02:47:39.114703
1694797,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1559880,TOPUP,0,17.90,1,NaN,INBOUND,2018-11-15 05:46:21.746703
1694802,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1679226,CARD_PAYMENT,0,12.08,0,TRUE,OUTBOUND,2018-11-15 05:51:34.235703
1694795,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1531550,CARD_PAYMENT,0,12.08,0,TRUE,OUTBOUND,2018-11-15 05:51:36.603703
1694803,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1679236,CARD_PAYMENT,0,12.07,1,TRUE,OUTBOUND,2018-11-15 05:55:27.831703
1694798,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1622625,TOPUP,0,0.18,2,NaN,INBOUND,2019-02-12 17:24:35.554703
1694801,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1647662,TOPUP,0,8.95,1,NaN,INBOUND,2019-02-12 17:24:50.538703
1694800,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1646384,CARD_PAYMENT,0,0.54,2,TRUE,OUTBOUND,2019-02-21 09:29:18.890703
1694799,user_12038,1958,SWEU,2018-10-18 02:44:25.713703,0,0,9,0,transaction_1646334,CARD_PAYMENT,0,17.01,1,FALSE,OUTBOUND,2019-02-21 09:30:21.218703


In [51]:
pd.crosstab(index=data['transactions_type'],columns=data['ea_cardholderpresence'])

ea_cardholderpresence,FALSE,TRUE,UNKNOWN
transactions_type,,,
ATM,73510,0,15
CARD_PAYMENT,932548,228607,4202
CARD_REFUND,29,0,2


In [54]:
data['num_contacts'].describe()

count    2.166096e+06
mean     2.671861e+01
std      5.554612e+01
min      0.000000e+00
25%      3.000000e+00
50%      1.300000e+01
75%      3.200000e+01
max      2.918000e+03
Name: num_contacts, dtype: float64

In [53]:
data['ea_cardholderpresence'].value_counts()

FALSE      1006087
TRUE        228607
UNKNOWN       4219
Name: ea_cardholderpresence, dtype: int64